# Qubit-waveguide coupling

In this notebook we are going to model the coupling of a superconducting qubit to a microwave guide. The model is quite generic: we will not care about the details of the transmon, replacing it with a two-level system, but we will model carefully (i) the discrete nature of the waveguide modes, and (ii) the inhomogeneity of the couplings between the qubit and the waveguide. The first feature appears in nano-photonic systems, but the second one is less relevant, for reasons that will become obvious below.

## Qubit-cavity coupling

Our starting point for this modelization, will be the interaction between a two-level system and a cavity. We will write down the Hamiltonian in the rotating-wave approximation (RWA) as
$$H = \frac{\Delta}{2}\sigma^z + \omega a^\dagger a + g (\sigma^+ a + \sigma^- a^\dagger).$$

Note that in this picture, the total number of excitations in the qubit and the cavity are conserved:
$$N := \frac{1}{2}(\sigma^z+1) + a^\dagger a$$
$$[H,N]=0$$

**Exercise 1:** Create a function that constructs the previous Hamiltonian as a sparse matrix, considering up to $n_{max}$ excitations in the cavity subspace.

When using two excitations, if you choose the right order, you should obtain something like
```
print(Hcqed(nmax=2).todense())
=>
[[ 0.5         0.          0.          0.          0.01        0.        ]
 [ 0.          1.5         0.          0.          0.          0.01414214]
 [ 0.          0.          2.5         0.          0.          0.        ]
 [ 0.          0.          0.         -0.5         0.          0.        ]
 [ 0.01        0.          0.          0.          0.5         0.        ]
 [ 0.          0.01414214  0.          0.          0.          1.5       ]]
```

**Hint 1:** To make a tensor product of operators $A \otimes B$ you can use `scipy.sparse.kron(A, B)`.

**Hint 2:** I like to write Greek letters for my constants. You can type them in Jupyter by writing the name in latex, like `\Delta`, and then pressing the `<TAB>` key. Use standard Latin names if you find this uncomfortable.

In [ ]:
import numpy as np
import scipy.sparse as sp

def Hcqed(Δ=1.0, ω=1.0, g=0.01, nmax=1):
    """Construct the sparse-matrix representation for a cavity-qubit interaction
    Hamiltonian in the RWA limit.
    
    Parameters
    ----------
    Δ    -- qubit energy gap
    ω    -- cavity frequency
    g    -- qubit-cavity coupling
    nmax -- maximum number of photons
    """
    
    pass # Replace with as many lines of code as you need

**Exercise 2:** Construct the excitation number operator and show that it is a conserved quantity. You can use the operator norm $\Vert{O}\Vert_\infty = \max(|O_{ij}|)$ to show that the commutator is zero.

In [ ]:
def Nexcitation(nmax=1):
    """Return the operator that represents the total number of excitations in
    the cavity and the qubit, truncated up to nmax photons"""
    
    pass # Replace with as many lines of code as you need

In [ ]:
def exercise2():
    H = Hcqued(nmax=10)
    N = Nexcitation(nmax=10)
    commutator = np.linalg.norm(H @ N - N @ H)
    print(f'Commutator norm: ||[H, N]||={np.max(np.abs(commutator))}')

exercise2()

## Qubit-waveguide coupling

Microwave cavities are typically made of thin cables, called coplanar waveguides, that are printed on the 2D chip. Other times, they are made of Copper or Aluminum tubes.

![Coplanar  waveguide](./figures/fig-coplanar-waveguide.png)

![3D waveguide](./figures/fig-3d-waveguide.png)

In all these cases, we have a situation in which there is a large number of electromagnetic modes that can be hosted in the waveguide. We can therefor expand observables such as the voltage, using the spatial distribution of those modes and one Fock operator per each mode
$$V(\mathbf{x}) = \sum_k u_k(\mathbf{x}) \frac{V_k}{\sqrt{2}}(a^\dagger_k + a_k).$$

We can add even more information. First, because the waveguides are tight, we can approximate
$$u_k(\mathbf{x}) \sim \sqrt{\frac{2}{d}} \cos\left(\frac{\pi n}{d} x\right) w(y,z),$$
with a common transverse profile $w(y,z)$ and equispaced momenta $k=\pi n/d$ that depend on the length of the waveguide $d$ and an integer $n=1,2,\ldots\in \mathbb{N}.$ Note that these modes satisfy that they are non-zero at the boundaries of the waveguide, as we expect from the accummulation of charge on the surface of the superconductor.

A second piece of information is that, because the modes of the waveguide behave as independent harmonic oscillators, the constants $V_k$ have a very precise dependence
$$V_k \propto \sqrt{\omega_k}$$
in terms of the frequency of the respective modes.

Finally, if we work with a coplanar waveguide, the growth of the frequencies is approximately linear
$$\omega_k \simeq v |k| = \frac{1}{\sqrt{c_0 l_0}} |k|,$$
and given by the capacitance $c_0$ and inductance $l_0$ per unit length.

When we place a qubit in such a waveguide, we obtain a generalization of the previous model
$$H_{wqed} = \frac{\Delta}{2}\sigma^z + \sum_k \omega_k a_k^\dagger a_k + \sum_k (g_k \sigma^+ a_k + \mathrm{H.c.}),$$
with constants
$$g_k \sim \frac{g}{\sqrt{d}} \cos(k x) \sqrt{\omega_k}.$$

Note how the couplings seem to decrease with the length of the waveguide $d.$ This is compensated by the fact that as we decrease $d$ the spacing between modes $v\pi/d$ is also reduced.

Instead of putting in microscopic models, we can paramterize everything in terms of
- The qubit's frequency $\Delta$
- The wavelength of the qubit in the waveguide $\lambda$
- The position of the qubit in units of this wavelength $u = x/\lambda$
- The coupling strength $g_{cav}$ between the qubit and a cavity of length $d=\lambda/2$ whose fundamental mode would have $\omega_1 = \Delta,$ with the qubit placed at $x=0.$
- The actual length of the waveguide measured in numbers of wavelengths $N_{\lambda} = 2d/\lambda.$

When we do so, we find that if we build that cavity of length $d=\lambda/2$, the coupling strength is
$$g_{cav} = \frac{g}{\sqrt{\lambda/2}}\sqrt{\Delta},$$
and the frequency is matched to the wavelength by
$$\Delta = v \frac{\pi}{\lambda}.$$
Hence, we can express the rescaled couplings and the frequency dispersion relation as
$$k_n = \frac{\pi}{\lambda} \frac{n}{N_\lambda},$$
$$g_{k_n} = g_{cav}\sqrt{\frac{\omega_{k_n}}{N_{\lambda}\Delta}}\cos\left(\frac{\pi n}{N_\lambda} u\right),$$
$$\omega_{k_n} = \Delta \frac{n}{N_\lambda}.$$

## Wigner-Weisskopf theory

As we saw with the cavity, in a RWA model the total number of excitations in the qubits and photon spaces is conserved
$$N = \frac{1}{2}(\sigma^z+1) + \sum_k a^\dagger_k a_k.$$
If we know the number of excitations is 0, the only available state is the vacuum
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}\ket{0,\mathrm{vac}}.$$
However, if we have one excitation, we can write down a Wigner-Weisskopf model
$$\ket{\Psi} := \psi_0\ket{1,\mathrm{vac}} + \sum_{n=1}^{n_{max}} \psi_n a_{k_n}^\dagger\ket{0,\mathrm{vac}} =: \sum_{n=0}^{n_{max}} \psi_n \ket{n}.$$

A nice thing about this representation is that we can project the $H_{wqed}$ model into a matrix of size $n_{max}+1,$ noting that
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}\newcommand{\ketbra}[2]{\ket{#1}\!\bra{#2}}
H_{wqed} \sim H_{WW} = \Delta\ketbra{0}{0} +  \sum_{n=1}^{n_{max}}\omega_{k_n}\ketbra{n}{n}
+ \sum_{n} (g_{k_n}\ketbra{0}{n}+\mathrm{H.c.}) - \frac{\Delta}{2}.$$

**Exercise 3:** Demonstrate the expression for the Wigner-Weisskopf Hamiltonian by projecting $H_{wqed}$ onto a single-excitation subspace. Write down a function that takes $\Delta, n_{max}, u_{qubit}$ and $N_\lambda$ and returns the Wigner-Weisskopf Hamiltonian in the form of a sparse matrix with $(n_{max}+1)\times(n_{max}+1)$ elements.

In [ ]:
def Hwigner(Δ, nmax, Nλ, uqubit=0):
    """Construct the Wigner-Weisskopf approximation for a qubit placed at
    position uqubit, in a waveguide that is Nλ wavelengths long, for a qubit
    with a gap Δ."""
    
    pass # Replace with as many lines of code as you need

**Exercise 4:** Simulate the dynamics of a qubit that is initially excited, with an empty waveguide
$$\ket{\Psi(0)} = \ket{1,\mathrm{vac}}$$
and evolves in time with the Wigner-Weisskopf model. Plot the dynamics as a function of time for different waveguide lengths $N_\lambda.$ At some point, the qubit should begin to decay exponentially in time, simulating the spontaneous emission on the waveguide. Estimate the decay rate as a function of $N_\lambda.$ Show that it converges and obtain is dependence with $g_{cav}.$

**Hint 1:** You can use Scipy's expm_multiply to obtain the evolution with a constant Hamiltonian, as
```
def decay(T, Δ, gcav, nmax, Nλ, uqbit=0, Nsteps=100):
    
    H = Hwigner(Δ, gcav, nmax, Nλ, uqbit)
    ψ0 = np.zeros(nmax+1)
    ψ0[0] = 1.0
    
    δt = T/Nsteps
    ψ = sp.linalg.expm_multiply(-1j * δt * H, ψ0, start=0, stop=T, num=Nsteps)
    
    return ψ
```

**Hint 2:** The decay rate should be stable with respect to $N_\lambda$ when the waveguide is long enough that the time for the photon to reach its end is smaller than the time for the qubit to decay. However, as you increase the size of the waveguide, you may need to also increase the number of modes in the description, so that the frequency of the qubit is still captured by the simulation
![Qubit decay](./figures/fig-decay.png)

# Challenge

We are going to study a problem with two qubits, one placed at the beginning $x=0$ and one at some other point along the waveguide. We will still work within the Wigner-Weisskopf approximation, assuming that there is a single excitation in either one of the qubits or as a propagating photon.

1. Derive the Wigner-Weisskopf Hamiltonian for the two qubits with the waveguide

2. Study the dynamics of a situation in which the qubit 1 is excited and it relaxes, producing an excitation that may (or may not) be absorbed by qubit 2.

3. Plot the dynamics for different positions of the qubit 2, measured in wavelengths. At which positions are the excitations perfectly exchanged? At which positions do the qubits ignore each other?

4. Make a 2D plot of the dynamics of the photon along the waveguide as the qubits exchange excitations. Use this to illustrate the existence of a causal cone and a propagation time proportional to $|u_{qubit 2} - u_{qubit 1}|$ at which the interaction between the qubits is activated.

Suggested reading: [Photon-mediated qubit interactions in one-dimensional discrete and continuous models, G. Díaz-Camacho et al, PRA 91, 063828 (2015)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.91.063828)